# TF-IDF e busca por conteúdo

Nesta atividade, vamos lidar com a seguinte situação: temos um grande banco de dados com textos, e queremos encontrar qual texto é mais relevante para uma consulta. Esse problema aparece em buscadores como Google, e também em sistemas locais como ElasticSearch.

In [1]:
import pandas as pd

DATASET = 'datasets/wikipedia_movies.zip'
df = pd.read_csv(DATASET).sample(1000)
df = df[['Title', 'Plot']]
print(df.head(), len(df))

                 Title                                               Plot
21521       Ex Machina  Programmer Caleb Smith, who works for the domi...
27887     Chanthupottu  Radhakrishnan (Dileep) is dysfunctionally brou...
19258   The Black Tent  The film begins with a tank battle where blond...
32143  Sandade Sandadi  Sandade Sandadi is a family drama based movie ...
4978         Red Light  Bookkeeper Nick Cherney is sent to jail for em... 1000


## Exercício 1
**Objetivo: lembrar-se do que é TF e o que é DF**

Identifique o Term Frequency e o Document Frequency nas asserções abaixo:

1. Quanto maior o <font color='red'>DF</font>, mais comum é a palavra entre os documentos de uma coleção
1. Quanto maior o <font color='red'>TF</font>, mais vezes a palavra é mencionada num documento específico
1. $P(w | \text{documento})$ -> <font color='red'>TF</font>
1. $P(w | \text{coleção})$ -> <font color='red'>DF</font>
1. Ajuda a identificar a coleção da qual um documento faz parte -> <font color='red'>DF</font>
1. Ajuda a identificar um documento dentro de uma coleção -> <font color='red'>TF</font>

## Exercício 2
**Objetivo: refletir sobre o uso de TF-IDF**

A medida TFIDF diz o quão relevante um documento é dentro de uma coleção e em relação a uma palavra específica. Ela é calculada para um par palavra-documento como:

$\text{TFIDF = TF / DF}$

Quando um documento tem um TFIDF alto em relação a uma palavra, isso significa que:

1. A palavra tende a ser (comum / incomum)
    - <font color=red>comum</font>


2. O documento menciona a palavras (muitas / poucas) vezes
    - <font color=red>muitas</font>

Portanto, qual seria uma maneira de escrever um documento que tem intencionalmente um TFIDF alto para uma palavra?

## Exercício 3
**Objetivo: calcular TFIDF para documentos usando sklearn**

TFIDF pode ser entendido como um processo de vetorização, semelhante a usar o CountVectorizer. Abaixo, há um código que mostra um exemplo dessa vetorização usando sklearn. 

1. Escolhendo um filme aleatório da coleção que carregamos, identifique o TFIDF das palavras "zombie", "fungus" e "survival".
1. Identifique o filme que tem o maior TFIDF para a palavra "zombie".

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['Plot'])

j = vectorizer.vocabulary_['test']
print(tfidf[2,j])

0.0


In [3]:
# 1. Escolhendo um filme aleatório da coleção que carregamos, identifique o TFIDF das palavras "zombie", "fungus" e "survival".
words = ['zombie', 'fungus', 'survival']
movie = np.random.randint(df['Title'].count())

print(f'No filme {df.iloc[movie]["Title"]}')
for word in words:
    try:
        j = vectorizer.vocabulary_[word] #indice da palavra no vocabulário
        print(word, tfidf[movie,j]) # idx do filme, idx da palavra. Retorna o TFIDF da palavra no documento.
    except:
        print(f'Não foi encontrado a palavra {word} no vocabulário')

No filme Munimji
zombie 0.0
Não foi encontrado a palavra fungus no vocabulário
survival 0.0


In [4]:
# 2. Identifique o filme que tem o maior TFIDF para a palavra "zombie".

word = 'zombie'
word_index = vectorizer.vocabulary_['zombie']

max_tfidf = -1
max_tfidf_movie = None

for movie in range(tfidf.shape[0]):
    if tfidf[movie, word_index] > max_tfidf:
        max_tfidf = tfidf[movie, word_index]
        max_tfidf_movie = movie

if max_tfidf_movie is not None:
    print(f'O filme com o maior TF-IDF para a palavra {word} é o filme {df.iloc[max_tfidf_movie]["Title"]}, id {max_tfidf_movie}')
else:
    print(f'')

O filme com o maior TF-IDF para a palavra zombie é o filme Flesheater, id 156


## Exercício 4
**Objetivo: implementar uma busca por vários termos simultaneamente**

Uma possível maneira de implementar uma busca por vários termos é somar o TFIDF de todas as palavras da query para cada documento da coleção, e então retornar o documento que tem a maior soma. Por exemplo, numa busca por "zombie fungus survival" deveríamos somar, para cada documento, o TFIDF de "zombie", de "fungus" e de "survival" e então ordenar o resultado.

1. Escreva código que implemente uma busca na base de dados de filmes à partir de uma query específica.
1. Qual é a complexidade ($O(...)$) da sua busca?

In [5]:
import numpy as np
query = "zombie fungus survival"

words = query.split()

# Calcula a soma do TF-IDF de cada termo para cada documento
scores = tfidf[:, [vectorizer.vocabulary_.get(word, -1) for word in words]].sum(axis=1)

# Ordena os documentos pelo score total decrescente
results = pd.DataFrame({'Title': df['Title'].to_numpy(), 'Score': list(scores)}).sort_values('Score', ascending=False)

# Imprime os títulos dos filmes encontrados
print(f"Resultados da busca por '{query}':")
for title in results['Title'].tolist()[:10]:
    print(title)

Resultados da busca por 'zombie fungus survival':
Flesheater
Warm Bodies
Being Caribou
Maniac
Coming Home
 Hounds of Love
Monster from Green Hell
The Story of G.I. Joe
Blackenstein
Mallu Singh (മല്ലൂസിംഗ്)


In [6]:
# Outra opção sem usar o DataFrame
results = {title:score.item() for title,score in zip(df['Title'].to_numpy(), list(scores))}

max_movie = max(results, key=results.get)
max_value = max(results.values())
print(f'O filme com maior TF-IDF para busca "{query}", é: {max_movie} com valor: {max_value}')

O filme com maior TF-IDF para busca "zombie fungus survival", é: Flesheater com valor: 0.19678560774236914


## Exercício 5
**Objetivo: implementar um índice invertido**

Você provavelmente reparou (talvez não tenha reparado, e é tudo bem) que, para fazer a busca, até agora, teve que varrer todos os documentos da sua coleção. Isso provavelmente levaria algum tempo, especialmente quando a coleção começa a aumentar. Para evitar ter que varrer todos os documentos da coleção, podemos implementar uma técnica chamada *índice invertido*. A ideia do índice invertido é usar um dicionário cujas chaves são as palavras do vocabulário e cujo conteúdo é uma lista de documentos que contém essa palavra, possivelmente acompanhados do TFIDF correspondente. Por exemplo:

In [7]:
indice = { 'palavra_1' : {'documento_1': 0.5, 'documento_2': 0.1, 'documento_3': 0.6}, 
          'palavra_2' : {'documento_2': 0.6},
         'palavra_3' : {'documento_2':0.3, 'documento_3': 0.2}
         }

def buscar(palavras, indice):
    assert type(palavras)==list
    resultado = dict()
    for p in palavras:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
                    
    return resultado

r = buscar(['palavra_1', 'palavra_2'], indice)
r

{'documento_1': 0.5, 'documento_2': 0.7, 'documento_3': 0.6}

1. Adicione uma nova palavra ao índice e escolha seu TFIDF. Realize uma nova busca e verifique o resultado.
1. Escreva uma função que ordena o resultado e retorna apenas `N` documentos mais relevantes para sua busca.
1. Incremente sua biblioteca de forma que ela passe a receber uma string como entrada (representando a query) e retorne os `N` documentos mais relevantes (`N` pode ser definido arbitrariamente).

In [8]:
# 2. Escreva uma função que ordena o resultado e retorna apenas N documentos mais relevantes para sua busca.
def n_relevantes(resultado, n):
    return sorted(resultado.items(), key=lambda x: x[1], reverse=True)[:n]

n_relevantes(r,2)

[('documento_2', 0.7), ('documento_3', 0.6)]

In [9]:
# 3. Incremente sua biblioteca de forma que ela passe a receber uma string como entrada (representando a query) e retorne 
# os N documentos mais relevantes (N pode ser definido arbitrariamente).

def n_relevantes_query(query, indice, n=2):
    palavras = query.split()
    busca = buscar(palavras, indice)
    rel = n_relevantes(busca, n)
    return rel

n_relevantes_query('palavra_1 palavra_2', indice)

[('documento_2', 0.7), ('documento_3', 0.6)]

## Exercício 6
**Objetivo: implementar um buscador de filmes**

Implemente uma função que recebe como entrada uma query e retorna os títulos e enredos dos 5 filmes mais relevantes para aquela query. Se precisar, use mais parâmetros ou variáveis globais. Teste a sua função e veja se você concorda com os resultados, incluindo se você consegue encontrar seus filmes favoritos e se consegue alguma recomendação relevante a um filme novo.

In [10]:
tfidf.shape[0]

1000

In [239]:
from tqdm import tqdm
#indice_filmes = {word:{i:tfidf[i,word_idx] for i in range(tfidf.shape[0])} for word, word_idx in tqdm(vectorizer.vocabulary_.items())}

indice_filmes = {}
for word, word_idx in tqdm(vectorizer.vocabulary_.items()):
    indice_filmes[word] = {}
    for i in range(tfidf.shape[0]):
        indice_filmes[word][i] = tfidf[i,word_idx]

100%|████████████████████████████████████████████████████████████████████████████| 24678/24678 [12:03<00:00, 34.11it/s]


In [240]:
def query_movies(query, indice):
    result = n_relevantes_query(query, indice, n=5)
    
    for i in result:
        print(df.iloc[i[0]]['Title'])
        print(df.iloc[i[0]]['Plot'])
        print('\n')

In [289]:
query = 'zombie fungus survival'
query_movies(query, indice_filmes)

Voodoo Man
Nicholas (George Zucco) runs a filling station in the sticks. In reality, he is helping Dr. Richard Marlowe (Bela Lugosi) capture comely young ladies so he can transfer their life essences to his long-dead wife. Also assisting is Toby (John Carradine), who lovingly shepherds the left-over zombie girls and pounds on bongos during voodoo ceremonies. The hero is a Hollywood screenwriter who, at the end of the picture, turns the experience into a script titled "Voodoo Man". When his producer asks who should star in it, the hero suggests ... Bela Lugosi.


The Precipice Game
Liu Chenchen, a free-spirited young woman, rebels against her wealthy family and elopes with her boyfriend to join a cruise-bond treasure hunt. But what began as an innocent game with promises of great reward soon turns into a battle for survival when the contestants are thrown into a mysterious world of intrigue and chaos in the middle of the sea. Liu relies only on her wits and her new friends to survive, a

## Exercício 7
**Objetivo: identificar palavras-chave usando TFIDF**

Uma outra aplicação de TFIDF é encontrar palavras-chave, isto é, palavras que diferenciam um documento do restante dos documentos de sua coleção.

Incremente seu buscador de forma que, além do título e enredo, ele também escolha as algumas palavras (escolha quantas!) mais relevantes de cada documento e as imprima como keywords.

In [329]:
def search_keywords(index):
    # Obtem os índices das N palavras com maior peso no documento, em ordem decrescente.
    keywords_list = np.argsort(tfidf[index,:].toarray())[0][::-1][:5]

    # Recuperar as palavras correspondentes aos índices encontrados 
    vocabulary = vectorizer.vocabulary_
    keywords = [list(vocabulary.keys())[list(vocabulary.values()).index(i)] for i in keywords_list]        
    return keywords

In [330]:
def query_movies(query, indice):
    result = n_relevantes_query(query, indice, n=5)
    
    for i in result:
        print(df.iloc[i[0]]['Title'], '-', i[0])
        print(f'Keywords: {search_keywords(i)}')
        print(df.iloc[i[0]]['Plot'])
        print('\n')

In [335]:
query = 'Family love'
query_movies(query, indice_filmes)

Hands Up - 361
Keywords: ['nagababu', 'cameo', 'fall', 'brahmanandam', 'heels']
Cops Nagababu and Brahmanandam fall for their newly transferred senior police official. In an interesting plot sequence, they fall head over heels in love. While fighting for love and against the goons, they learn she is married (to Chiranjeevi in cameo).


Nee Kosam - 841
Keywords: ['ravi', 'sasi', 'love', 'is', 'maheswari']
Sasirekha (Maheswari) is the daughter of a rich father (Jayaprakash Reddy), who is a strict disciplinarian and hates any kind of love affair. Ravi (Ravi Teja), an orphan falls in love with Sasi and she also slowly develops feelings for Ravi. One day Ravi meets Sasi’s father in the outskirts of the city and tells him all about his love. However, he is accidentally killed by Ravi in a scuffle. Sasi soon finds out that the real culprit is Ravi. Infuriated, she breaks-up with him. Ravi, unable to bear the rejection, jumps from a hill. He escapes death, only to be told by the doctors that h

## Exercício 8
**Objetivo: encontrar documentos semelhantes usando TFIDF**

Uma maneira de encontrar documentos semelhantes em uma coleção de textos é assumir que o texto do documento é uma query, e então realizar a busca normalmente. O problema disso é que provavelmente teríamos textos muito longos e a query ficaria muito carregada. Para solucionar isso, poderíamos usar apenas as palavras mais relevantes de um documento como query. Implemente uma função que recebe o índice (ou outro identificador único) de um documento de nosso banco de dados e então encontra 5 documentos semelhantes a ele.

In [332]:
def search_similar(index, indice):
    query = ' '.join(search_keywords(index))
    return query_movies(query, indice_filmes)

In [336]:
search_similar(23, indice_filmes)

Dangerous Corner - 23
Keywords: ['robert', 'ann', 'martin', 'bonds', 'betty']
Robert Chatfield is having dinner with his wife, Freda, and four of their friends: Charles Stanton, his business partner; Ann Peel, who works at their company; and Robert's sister, Betty, and her husband, Gordon, who is another partner in the firm. As the dinner winds down, the subject of Robert's brother's suicide the prior year comes up. Robert's brother, Martin, had died from a gunshot wound, which an investigation had ruled a suicide, brought on by his guilt over stealing some bonds from their company, of which he was also a partner. But now, during their dinner conversation, certain comments made by his companions don't add up in Robert's mind.
As he begins to question them, Freda confesses that she had been secretly in love with Martin, and Ann reveals that she has been holding a torch for Robert for years. It was this unspoken love which caused Ann to not speak honestly at the hearing into Martin's dea